<a href="https://colab.research.google.com/github/devcon14/colab-connect/blob/main/backtests/Backtest_Smash_Days.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

| Strategy   | Profit Factor | Win Rate | Return | Drawdown |
|------------|---------------|----------|--------|----------|
| Smash Days | 1.577189      |          | 218%   | 18%      |


In [24]:
# !pip install quandl
if True:
  !rm -rf colab-connect
  !git clone https://github.com/devcon14/colab-connect.git
  import sys
  if not "colab-connect" in sys.path:
    sys.path.append("colab-connect")

  # from dataloader import gather_gsheets, gather_av
  from features_kaggle import win_streak_nb, loss_streak_nb
  # get_features, 
  from features_main import get_features_cnd # , get_features_ta
  # from dataloader import get_drive_csv, get_av_api

ALPHAVANTAGE_API = "C8RFYYD13O6DPWQ2"

Cloning into 'colab-connect'...
remote: Enumerating objects: 167, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 167 (delta 6), reused 0 (delta 0), pack-reused 152
Receiving objects: 100% (167/167), 2.11 MiB | 6.52 MiB/s, done.
Resolving deltas: 100% (87/87), done.


In [25]:
import pandas_datareader.data as web
import pandas as pd

symbol = "SPY"

if "/" in symbol:
  sec = web.DataReader(symbol, "av-forex-daily", api_key=ALPHAVANTAGE_API)
else:
  # sec = web.DataReader(symbol, "av-daily-adjusted", api_key=ALPHAVANTAGE_API)
  sec = web.DataReader(symbol, "av-daily-adjusted", api_key=ALPHAVANTAGE_API, start="2010-01-01")
  # sec["close"] = sec["adjusted close"]

sec["symbol"] = symbol
sec

,open,high,low,close,adjusted close,volume,dividend amount,split coefficient,symbol
2010-01-04,112.37,113.3900,111.51,113.33,88.390196,118857369,0.0,1.0,SPY
2010-01-05,113.26,113.6800,112.85,113.63,88.624176,111519230,0.0,1.0,SPY
2010-01-06,113.52,113.9900,113.43,113.71,88.686571,116017127,0.0,1.0,SPY
2010-01-07,113.50,114.3300,113.18,114.19,89.060941,130502017,0.0,1.0,SPY
2010-01-08,113.89,114.6200,113.66,114.57,89.357317,126357911,0.0,1.0,SPY
...,...,...,...,...,...,...,...,...,...
2022-12-19,383.47,383.8200,378.28,380.02,380.020000,79878103,0.0,1.0,SPY
2022-12-20,379.23,382.2300,377.85,380.54,380.540000,74427240,0.0,1.0,SPY
2022-12-21,383.25,387.4100,382.69,386.23,386.230000,78167369,0.0,1.0,SPY
2022-12-22,383.05,386.2134,374.77,380.72,380.720000,100120855,0.0,1.0,SPY


In [26]:
from numba import jit
from features_kaggle import win_streak_nb, get_features, loss_streak_nb

# ft_cnd_closing_low_1
get_features_cnd(sec)
# get_features_pa(sec)
# get_features_ts(sec)
# get_features_ta(sec)

sec["ft_cnd_win_streak"] = sec["close"].rolling(10).apply(win_streak_nb, engine="numba", raw=True)
sec["ft_cnd_loss_streak"] = sec["close"].rolling(10).apply(loss_streak_nb, engine="numba", raw=True)

sec["ft_cnd_green"] = sec.close > sec.open
sec["ft_cnd_red"] = sec.close <= sec.open
sec["ft_cnd_session_return"] = sec.close - sec.open

sec

,open,high,low,close,adjusted close,volume,dividend amount,split coefficient,symbol,ft_cnd_range,...,tmp_low_1,ft_cnd_inside_bar,ft_cnd_engulfing,ft_cnd_green_soldier,ft_cnd_red_soldier,ft_cnd_win_streak,ft_cnd_loss_streak,ft_cnd_green,ft_cnd_red,ft_cnd_session_return
2010-01-04,112.37,113.3900,111.51,113.33,88.390196,118857369,0.0,1.0,SPY,1.8800,...,NaN,False,False,False,False,NaN,NaN,True,False,0.96
2010-01-05,113.26,113.6800,112.85,113.63,88.624176,111519230,0.0,1.0,SPY,0.8300,...,111.51,False,False,True,False,NaN,NaN,True,False,0.37
2010-01-06,113.52,113.9900,113.43,113.71,88.686571,116017127,0.0,1.0,SPY,0.5600,...,112.85,False,False,True,False,NaN,NaN,True,False,0.19
2010-01-07,113.50,114.3300,113.18,114.19,89.060941,130502017,0.0,1.0,SPY,1.1500,...,113.43,False,True,False,False,NaN,NaN,True,False,0.69
2010-01-08,113.89,114.6200,113.66,114.57,89.357317,126357911,0.0,1.0,SPY,0.9600,...,113.18,False,False,True,False,NaN,NaN,True,False,0.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-19,383.47,383.8200,378.28,380.02,380.020000,79878103,0.0,1.0,SPY,5.5400,...,381.04,False,False,False,True,0.0,4.0,False,True,-3.45
2022-12-20,379.23,382.2300,377.85,380.54,380.540000,74427240,0.0,1.0,SPY,4.3800,...,378.28,False,False,False,True,1.0,0.0,True,False,1.31
2022-12-21,383.25,387.4100,382.69,386.23,386.230000,78167369,0.0,1.0,SPY,4.7200,...,377.85,False,False,True,False,2.0,0.0,True,False,2.98
2022-12-22,383.05,386.2134,374.77,380.72,380.720000,100120855,0.0,1.0,SPY,11.4434,...,382.69,False,False,False,True,0.0,1.0,False,True,-2.33


In [27]:
try:
  from backtesting import Backtest, Strategy
except Exception as e:
  !pip install backtesting

from backtest_lib_stop_entry import get_bt_results
from backtest_lib_stop_entry import get_ft_level, bt_convert_frame

if False:
  # utility functions
  def get_ft_level(price_level, signal_array, index=None):
    level = np.where(signal_array, price_level, None)
    level = pd.Series(level)
    level = level.ffill()
    if not index is None:
      # assigning failed where the dataframe has a date index
      # and the new series has a rangeindex
      # this allows for overriding the index
      level.index = index
    return level

  def bt_convert_frame(sec):
    # prepare dataframe for backtest
    # sec.rename(columns={"open": "Open", "close": "Close", "high": "High", "low": "Low", "volume": "Volume"})
    bt_df = sec.copy()
    bt_df.columns = [col[0].upper() + col[1:] for col in bt_df.columns]
    bt_df.index = pd.to_datetime(bt_df.index)
    return bt_df


In [28]:
import numpy as np

sec["ft_bullish_smash_day"] = sec["ft_cnd_closing_low"]
sec["ft_bearish_smash_day"] = sec["ft_cnd_closing_high"]

sec["ft_smash_day_high"] = get_ft_level(sec["high"], sec["ft_bullish_smash_day"], sec.index)
sec["ft_smash_day_low"] = get_ft_level(sec["low"], sec["ft_bearish_smash_day"], sec.index)

level = get_ft_level(sec["high"], sec["ft_bullish_smash_day"])

sec[["ft_smash_day_high", "ft_bullish_smash_day"]]


,ft_smash_day_high,ft_bullish_smash_day
2010-01-04,NaN,False
2010-01-05,NaN,False
2010-01-06,NaN,False
2010-01-07,NaN,False
2010-01-08,NaN,False
...,...,...
2022-12-19,383.8200,True
2022-12-20,383.8200,False
2022-12-21,383.8200,False
2022-12-22,386.2134,True


In [29]:
sec["signal_long"] = sec["ft_bullish_smash_day"]
sec["signal_short"] = sec["ft_bearish_smash_day"]


In [30]:
# from backtest_lib_stop_entry import StopEntryStrategy

# bt = get_bt_results(sec, StrategyClass=StopEntryStrategy)
bt = get_bt_results(sec)
bt._results

Start                     2010-01-04 00:00:00
End                       2022-12-23 00:00:00
Duration                   4736 days 00:00:00
Exposure Time [%]                   49.724602
Equity Final [$]                  21779.84092
Equity Peak [$]                   21779.84092
Return [%]                         117.798409
Buy & Hold Return [%]              237.871702
Return (Ann.) [%]                    6.186148
Volatility (Ann.) [%]               11.285984
Sharpe Ratio                         0.548127
Sortino Ratio                        0.897532
Calmar Ratio                         0.341532
Max. Drawdown [%]                  -18.112944
Avg. Drawdown [%]                   -2.902548
Max. Drawdown Duration     1483 days 00:00:00
Avg. Drawdown Duration       92 days 00:00:00
# Trades                                  403
Win Rate [%]                        85.856079
Best Trade [%]                       5.758922
Worst Trade [%]                     -8.903187
Avg. Trade [%]                    

In [31]:
if False:
  opt = bt.optimize(
      exit_delay=range(1,10),
      max_delay=range(1,20),
      constraint=lambda x: x["max_delay"] > x["exit_delay"]
  )
  display(opt._strategy)

Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

<Strategy StopEntryStrategy(exit_delay=1,max_delay=13)>